# WASD

# Actors
    CARLA defines actors as anything that plays a role in the simulation or can be moved around. 
    
    That includes: pedestrians, vehicles, sensors and traffic signs (considering traffic lights as part of these). 
    
    Actors are spawned in the simulation by carla.World and they need for a carla.ActorBlueprint to be created. 

In [1]:
!pip install carla

In [1]:
import carla
import math
import random
import time

# client : requires two arguments
    first : ip address of the machine where you are running carla server, in this case, my local machine.
    second : the port on which carla server is running, 2000 is the default

In [2]:
client=carla.Client('localhost', 2000)

# get_world
        
    allows to access all within the simulation so accessible vehicles, buildings, pedestrians, and other things in the map such as trees, traffic lights
        

In [3]:
# it also allows us to add or spawn these objects

world=client.get_world()

# The Blueprint Library (carla.BlueprintLibrary) is a summary of all carla.ActorBlueprint 
    blueprints = [bp for bp in world.get_blueprint_library().filter('*')]
    
    These layouts allow the user to smoothly incorporate new actors into the simulation.
    They are already-made models with animations and a series of attributes.
    
    These attributes include vehicle color, amount of channels in a lidar sensor, a walker's speed, and much more

In [4]:
# the blue print includes vehicles, boxes, kiosk or things you might find in the street, pedestrians

bp_lib=world.get_blueprint_library()

# Spawning
    The world object is responsible of spawning actors and keeping track of these. 
    Spawning only requires a blueprint, and a carla.Transform stating a location and rotation for the actor.

# spawn_actor(self, blueprint, transform, attach_to=None, attachment=Rigid) : 
    shows some sensors being attached to a car when spawned.
    
    Spawns an actor into the world based on the blueprint provided and the transform.

    spawn_actor() raises an exception if the spawning fails.
    try_spawn_actor() returns None if the spawning fails.

# get_map
    3D location which are set in sensible locations on the road so you can spawn your vehicles on the road that is pointing in the right direction.
    
    Returns a list of recommendations made by the creators of the map to be used as spawning points for the vehicles. 
    
    The list includes carla.Transform objects with certain location and orientation. 
    
# get_spawn_points
    Returns a list of recommendations made by the creators of the map to be used as spawning points for the vehicles. The list includes carla.Transform objects with certain location and orientation.

In [5]:
spawn_points=world.get_map().get_spawn_points()

In [6]:
# find the vehicle blueprint from the blueprint library
# lets specifying a specific vehicle which I want to use 'lincoln mkz_2020'

vehicle_bp=bp_lib.find('vehicle.lincoln.mkz_2020')

In [7]:
# using the 'world object' to 'try_spawn_actor' method to spawn that specific vehicle
# because when you spawn vehicles within simulation, it's not always necessarily going to work
# Try spawning the vehicle at a randomly chosen spawn point

vehicle=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# carla.Transform(self, location, rotation) 
    Class that defines a transformation, a combination of location and rotation, without scaling.
    
# transform(self, in_point)
    Translates a 3D point from local to global coordinates using the current transformation as frame of reference.
    
    in_point(carla.Location) : Location in the space to which the transformation will be applied.
    
# spectator
     An in-game point of view. It can be used to move the view of the simulator window. 
    
# get_spectator(self) 
    Returns the spectator actor. The spectator is a special type of actor created by Unreal Engine, usually with ID=0, that acts as a camera and controls the view in the simulator window.
    
# set_transform(self, transform)
    Teleports the actor to a given transform (location and rotation).
    
# get_transform(self)
    Returns the actor's transform (location and rotation) the client recieved during last tick.

In [8]:
# lets move the vehicle to the specific location which is going to take us just above and behind the car that I've just spawned

spectator=world.get_spectator()

# it will move the spectator to see right behind the vehicle that I just spawned so that I can see it 

transform=carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)



In [9]:
# add some more vehicles, randomly select vehicles from the blueprint library
for i in range(30):
    vehicle_bp=random.choice(bp_lib.filter('vehicle')) 
    npc=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# get_actors(self, actor_ids=None) :
    Retrieves a list of carla.Actor elements, either using a list of IDs provided or just listing everyone on stage. If an ID does not correspond with any actor, it will be excluded from the list returned, meaning that both the list of IDs and the list of actors may have different lengths.

    Parameters:
    actor_ids (list) - The IDs of the actors being searched. By default it is set to None and returns every actor on scene.

# set_autopilot(self, enabled=True, port=8000) :
    Registers or deletes the vehicle from a Traffic Manager's list. When True, the Traffic Manager passed as parameter will move the vehicle around. The autopilot takes place client-side.
    Parameters:
    enabled (bool)
    port (uint16) - The port of the TM-Server where the vehicle is to be registered or unlisted. If None is passed, it will consider a TM at default port 8000.

In [10]:
# by this, it will hand over control of each vehicles to the traffic manager and they will move around the city

for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [11]:
# recording data using sensors

camera_bp=bp_lib.find('sensor.camera.rgb')
camera_init_trans=carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) #record the data

### listen(self, callback) 
### The function the sensor will be calling to every time a new measurement is received.

In [12]:
# start the camera saving data to disk
cc = carla.ColorConverter.CityScapesPalette
camera.listen(lambda image:image.save_to_disk('output/%06d.png' % image.frame, cc))

In [18]:
# Stop the camera when we've recorded enough data
camera.stop()

In [14]:
# client.start_recorder("recording01.log")

In [15]:
# client.stop_recorder()

In [16]:
# client.show_recorder_file_info("recording01.log")

In [17]:
# client.replay_file("./carla/Unreal/CarlaUE4/Saved/recording01.log")

In [19]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()